In [16]:
class Data(object):
        def __init__(self):
            self._x_train = 'X_train.csv'
            self._x_test  = 'X_test.csv'
            self._y_train = 'y_train.csv'
            self._y_test  = 'y_test.csv'

        @property
        def x_train_file(self):
            return self._x_train

        @property
        def y_train_file(self):
            return self._y_train

        @property
        def x_test_file(self):
            return self._x_test

        @property
        def y_test_file(self):
            return self._y_test
        @property
        def date_column(self):
            return ['click_time']
        @property
        def test_full(self):
            return None

data= Data()

In [2]:
import pandas as pd

In [3]:
def read_data():
    X_train= pd.read_csv(data.x_train_file,parse_dates=data.date_column)
    X_test = pd.read_csv(data.x_test_file ,parse_dates=data.date_column)
    y_train= pd.read_csv(data.y_train_file)
    y_test = pd.read_csv(data.y_test_file )           
    return X_train,X_test,y_train, y_test

In [14]:
X_train_raw,X_test_raw,y_train_raw, y_test_raw = read_data()
X_train_raw.head()

,ip,app,device,os,channel,click_time,is_attributed
0,106857,19,6,29,213,2017-11-07 01:02:24,1
1,75754,19,1699,24,213,2017-11-08 06:22:48,1
2,109837,2,1,13,435,2017-11-08 04:00:23,0
3,248392,19,0,21,213,2017-11-08 15:41:55,1
4,100212,1,2,49,120,2017-11-09 08:18:11,0


In [5]:
import datetime
def datetime_to_float(a):
    d = None
    if(isinstance(a, str)):
        d=datetime.datetime.strptime(a, "%Y-%m-%d %H:%M:%S")
    elif(isinstance(a, pd.DatetimeIndex)):
        d=a
    else:
        return 0.0        
    epoch = datetime.datetime.utcfromtimestamp(0)
    total_seconds =  (d - epoch).total_seconds()
    return total_seconds

date_transformer = lambda x: datetime_to_float(x)
assert(datetime_to_float('2017-11-07 09:35:17') == 1510047317)
assert(datetime_to_float(pd.DatetimeIndex([pd.to_datetime('2017-11-07 09:35:17')])) == 1510047317)

In [6]:
def preprocess(data):
    data['click_time']=data['click_time'].map(date_transformer)
    if 'attributed_time' in data.columns:
        data['attributed_time']=data['attributed_time'].map(date_transformer)
    return data


In [7]:
X_train=preprocess(X_train_raw)
y_train=y_train_raw

In [8]:
input_dim=X_train.shape[1]

In [9]:
def get_model():
    import tensorflow as tf
    from keras.models import Sequential
    from keras.layers.core import Dense
    from keras.layers import Activation
    model = Sequential()
    model.add(Dense(100, input_dim=input_dim, activation='relu'))
    model.add(Dense(70, activation='relu'))
    model.add(Dense(35, activation='relu'))
    model.add(Dense(17, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [10]:
model = get_model()

Using TensorFlow backend.


In [11]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics = ['accuracy'])

In [12]:
y_train.head()

,is_attributed
0,1
1,1
2,0
3,1
4,0


In [13]:
model.fit(X_train,y_train,epochs=2)

Epoch 1/2
2489/2489 [==============================] - 0s 158us/step - loss: 8.3065 - acc: 0.4841
Epoch 2/2
2489/2489 [==============================] - 0s 87us/step - loss: 8.2630 - acc: 0.4873


In [15]:
def evaluate(X_test_raw,y_test_raw):
    X_test=preprocess(X_test_raw)
    y_test=y_test_raw
    return model.evaluate(X_test, y_test, verbose=0)
evaluate(X_test_raw,y_test_raw)

[7.8550622730477357, 0.51265568487037849]